# Session 13: Pivot tables. Merge, join, and concat.

## Pivot tables

Just like with Excel, we can use a similar set of operations to be performed on our dataframes:

The syntax for pivot tables in `pandas` is the following:

```Python
pd.pivot_table(
    data="our dataframe",
    values="column to aggregate",
    index="column to use as index",
    columns="column with categories with which to aggregate",
    aggfunc="function to use",
)
```

Let's see it with an example!

In [3]:
import pandas as pd

energy = pd.read_csv("../files/energy.csv")

energy.head()

,datetime,power_demand,nuclear,gas,solar,hydro,coal,wind,spot_price,year,month,day,hour,weekday
0,2018-12-31 23:00:00+00:00,23251.2,6059.2,2954.0,7.1,3202.8,1867.0,3830.3,66.88,2018,12,31,23,0
1,2019-01-01 00:00:00+00:00,22485.0,6059.2,3044.1,8.0,2884.4,1618.0,3172.1,66.88,2019,1,1,0,1
2,2019-01-01 01:00:00+00:00,20977.0,6059.2,3138.6,7.5,1950.8,1535.3,2980.5,66.00,2019,1,1,1,1
3,2019-01-01 02:00:00+00:00,19754.2,6059.2,3596.2,7.5,1675.7,1344.0,2840.0,63.64,2019,1,1,2,1
4,2019-01-01 03:00:00+00:00,19320.6,6063.4,3192.6,7.5,1581.8,1345.0,3253.4,58.85,2019,1,1,3,1


In [5]:
# Create a table with:
# months as rows -> `index`
# weekday as columns -> `columns`
# containing the average (`aggfunc`) solar power (`values`) per weekday and month
pt = pd.pivot_table(
    data=energy, 
    values="solar",
    index="month",
    columns="weekday",
    aggfunc="mean"
)

pt

weekday,0,1,2,3,4,5,6
month,,,,,,,
1,1176.332759,885.727848,848.296341,882.874286,1054.105000,846.781333,975.469697
2,1163.607463,1285.191176,1286.626761,1372.796970,1390.606780,1249.454688,1119.097143
3,1409.240000,1268.456164,1304.989041,1693.428125,1575.851852,1585.824390,1264.380851
4,1238.181395,1400.160674,1190.424324,1391.480000,1356.346154,1456.027273,1281.421739
5,1726.642254,1635.915068,1558.343011,1509.131915,1656.439286,1707.271429,1709.371233
6,1717.997101,1657.186301,1713.812676,1695.462857,1707.441667,1750.530769,1747.397674
7,1961.660000,1841.902299,1878.618824,1991.727692,1894.807576,1835.949231,1947.832308
8,1784.832353,1851.353030,1689.774286,1903.604762,1930.575294,1813.912222,1955.343077
9,1648.383146,1658.422857,1896.255738,1620.765714,1396.293056,1567.959677,1484.169231


In [7]:
# to check let's calculate the average solar power generation 
# on Sundays in December
energy[
    (energy["weekday"]==6) &
    (energy["month"]==12)
]["solar"].mean()

# nice!

860.0474226804124

In the end, `pd.pivot_table` is nothing but filling a table with:

```Python
for idx in index:
    for category in column:
        table[idx, category] = df[
            (df[index]==idx) &
            (df[column]==category)
        ][values].aggfunc()
```

In [8]:
# Create a table with months and weekdays as rows
# containing the average solar power per weekday and month
pd.pivot_table(
    data=energy, 
    values="solar",
    index=["month", "weekday"],
    aggfunc="mean"
)

solar
month weekday             
1     0        1176.332759
      1         885.727848
      2         848.296341
      3         882.874286
      4        1054.105000
...                    ...
12    2         775.440476
      3         618.424419
      4         680.612941
      5         785.815909
      6         860.047423

[84 rows x 1 columns]

### Summarizing data

Pivot tables allow us to summarize data: 
* Create a table expressing the probability of having low/high wind in day of the week

In [11]:
# create categories of low/high wind
import numpy as np

energy["wind_cat"] = np.where(
    energy["wind"] > energy["wind"].mean(),
    1,
    0
)

# create table 
energy.pivot_table(
    index="weekday", columns="wind_cat", aggfunc="size",
)

wind_cat,0,1
weekday,,
0,706,543
1,729,519
2,663,585
3,705,543
4,775,473
5,776,472
6,708,540


In [12]:
# we convert times into probabilities by dividing by the length of the df!

energy.pivot_table(
    index="weekday", columns="wind_cat", aggfunc="size",
) / len(energy)

wind_cat,0,1
weekday,,
0,0.080806,0.062149
1,0.083438,0.059403
2,0.075884,0.066957
3,0.080691,0.062149
4,0.088703,0.054138
5,0.088818,0.054023
6,0.081035,0.061806


In [13]:
# Calculate the probabilities of having low/high wind for both weekdays and hours

energy.pivot_table(
    index=["weekday", "hour"], columns="wind_cat", aggfunc="size",
) / len(energy)

wind_cat             0         1
weekday hour                    
0       0     0.003205  0.002747
        1     0.003090  0.002861
        2     0.003319  0.002632
        3     0.003434  0.002518
        4     0.003434  0.002518
...                ...       ...
6       19    0.002861  0.003090
        20    0.002632  0.003319
        21    0.002747  0.003205
        22    0.002861  0.003090
        23    0.003205  0.002747

[168 rows x 2 columns]

## Merge, join and concat.

* `pd.join`: database-style join operation on the `index` labels of both dataframes
* `pd.merge`: like `pd.join` but extending functionalities
* `pd.concat`: puts together dataframes on the vertical (axis=0) or horizontal (axis=1)

### Merge

In [14]:
df_left = pd.DataFrame({
    "date": ["2020-01-01", "2020-01-01", "2020-01-02", "2020-01-02"],
    "hour": [1, 2, 1, 3],
    "price": [50, 51, 45, 47]
})

df_right = pd.DataFrame({
    "date": ["2020-01-01", "2020-01-01", "2020-01-02", "2020-01-02"],
    "hour": [3, 2, 1, 2],
    "sales": [46, 45, 15, 16]
})

In [15]:
df_left

,date,hour,price
0,2020-01-01,1,50
1,2020-01-01,2,51
2,2020-01-02,1,45
3,2020-01-02,3,47


In [16]:
df_right

,date,hour,sales
0,2020-01-01,3,46
1,2020-01-01,2,45
2,2020-01-02,1,15
3,2020-01-02,2,16


In [17]:
# outer join
pd.merge(df_left, df_right, on=["date", "hour"], how="outer")

,date,hour,price,sales
0,2020-01-01,1,50.0,NaN
1,2020-01-01,2,51.0,45.0
2,2020-01-02,1,45.0,15.0
3,2020-01-02,3,47.0,NaN
4,2020-01-01,3,NaN,46.0
5,2020-01-02,2,NaN,16.0


In [18]:
# left join
pd.merge(df_left, df_right, on=["date", "hour"], how="left")

,date,hour,price,sales
0,2020-01-01,1,50,NaN
1,2020-01-01,2,51,45.0
2,2020-01-02,1,45,15.0
3,2020-01-02,3,47,NaN


In [19]:
# right join
pd.merge(df_left, df_right, on=["date", "hour"], how="right")

,date,hour,price,sales
0,2020-01-01,3,NaN,46
1,2020-01-01,2,51.0,45
2,2020-01-02,1,45.0,15
3,2020-01-02,2,NaN,16


In [20]:
# inner join
pd.merge(df_left, df_right, on=["date", "hour"], how="inner")

,date,hour,price,sales
0,2020-01-01,2,51,45
1,2020-01-02,1,45,15


In [21]:
# performing merge on several dataframes
df_extra = pd.DataFrame({
    "date": ["2020-01-01", "2020-01-01", "2020-01-02", "2020-01-02"],
    "hour": [1, 2, 1, 3],
    "temp": [17, 28, 4, 16]
})

# create list of dataframes to merge
dfs = [df_left, df_right, df_extra]

# using reduce from functools
from functools import reduce

reduce(lambda left, right: pd.merge(left, right, on=["date", "hour"], how="inner"), dfs)

,date,hour,price,sales,temp
0,2020-01-01,2,51,45,28
1,2020-01-02,1,45,15,4


### Concat

In [22]:
df1 = pd.DataFrame(
    data={
        "val1": [1, 2, 3],
        "val2": ["H", "J", "K"]
    },
    index=["a", "b", "c"]
)

df2 = pd.DataFrame(
    data={
        "val1": [5, 6, 7],
        "val2": ["L", "M", "N"]
    },
    index=["e", "g", "i"]
)

df3 = pd.DataFrame(
    data={
        "val3": [5, 6, 7],
        "val4": ["L", "M", "N"]
    },
    index=["a", "b", "c"]
)

In [23]:
df1

,val1,val2
a,1,H
b,2,J
c,3,K


In [24]:
df2

,val1,val2
e,5,L
g,6,M
i,7,N


In [25]:
df3

,val3,val4
a,5,L
b,6,M
c,7,N


In [26]:
# with same names of columns
pd.concat([df1, df2])

,val1,val2
a,1,H
b,2,J
c,3,K
e,5,L
g,6,M
i,7,N


In [35]:
# by default, axis=0
pd.concat([df1, df3])

,val1,val2,val3,val4
a,1.0,H,NaN,NaN
b,2.0,J,NaN,NaN
c,3.0,K,NaN,NaN
a,NaN,NaN,5.0,L
b,NaN,NaN,6.0,M
c,NaN,NaN,7.0,N


In [36]:
# concat on axis=1 (VERTICALLY) 

# with same index
pd.concat([df1, df3], axis=1)

,val1,val2,val3,val4
a,1,H,5,L
b,2,J,6,M
c,3,K,7,N


In [37]:
# with different index
df2.index = ["x", "y", "z"]

pd.concat([df1, df2], axis=1, sort=False)

,val1,val2,val1,val2
a,1.0,H,NaN,NaN
b,2.0,J,NaN,NaN
c,3.0,K,NaN,NaN
x,NaN,NaN,5.0,L
y,NaN,NaN,6.0,M
z,NaN,NaN,7.0,N


## Practice

### Exercise 1:
Load all the `animals_YYYY.csv` datasets into a single dataframe called `animals`.

Create an extra column on each DF representing the year from the name.

In [46]:
animals_files = []

for file in os.listdir("../files"):
    if "animals_" in file:
        df = pd.read_csv(f"../files/{file}")
        df["year"] = int(file.split("_")[1].split(".")[0])
        animals_files.append(df)
    
animals = pd.concat(animals_files)

animals
    


,year,area,dogs,cats
0,2016,ARGANZUELA,10591,3202
1,2016,BARAJAS,5173,981
2,2016,CARABANCHEL,19275,3728
3,2016,CENTRO,15470,6164
4,2016,CHAMARTÍN,11759,2809
...,...,...,...,...
16,2018,TETUÁN,12478,4763
17,2018,USERA,12475,2419
18,2018,VICÁLVARO,5393,1279
19,2018,VILLA DE VALLECAS,9857,2467


### Exercise 2:
Create a table with average number of dogs and cats per area for the period 2015-2017

In [51]:
pd.pivot_table(
    animals[(animals["year"]>=2015) & (animals["year"]<=2017)],
    index="area",
    values=["dogs", "cats"],    
    aggfunc="mean"
)

,cats,dogs
area,,
ARGANZUELA,2936.333333,10396.000000
BARAJAS,902.666667,5220.000000
CARABANCHEL,3462.666667,19090.000000
CENTRO,5485.333333,14916.666667
CHAMARTÍN,2597.333333,12270.666667
CHAMBERÍ,2780.666667,13813.666667
CIUDAD LINEAL,5413.000000,17746.333333
FUENCARRAL-EL PARDO,3954.333333,18194.000000
HORTALEZA,5958.333333,16632.333333


### Exercise 3:
With the dataset `parks_and_extension.csv`, 
calculate the total extension of parks per area and sort them from highest to lowest.
* Check the formats!

In [60]:
parks = pd.read_csv("../files/parks_and_extension.csv", decimal=",")       

total_extension_per_area = pd.pivot_table(
    parks,
    index="area",
    values="extension",
    aggfunc="sum"
).sort_values(by="extension", ascending=False)

total_extension_per_area

,extension
area,
LATINA,79.04
VICÁLVARO,72.55
PUENTE DE VALLECAS,72.31
USERA,63.99
ARGANZUELA,44.84
HORTALEZA,42.30
FUENCARRAL - EL PARDO,39.94
TETUÁN,37.65
CIUDAD LINEAL,33.10


### Exercise 4:
What's the area with the most parks?

In [61]:
total_extension_per_area["extension"].idxmax()

'LATINA'

### Exercise 5:
What's the area with the biggest parks (on average)?

In [64]:
avg_extension_per_area = pd.pivot_table(
    parks,
    index="area",
    values="extension",
    aggfunc="mean"
).sort_values(by="extension", ascending=False)

avg_extension_per_area["extension"].idxmax()

'USERA'